# Etherscan verified contract database search

# Concatenate databases

In [3]:
import csv
import os
import glob
import pandas as pd
import tqdm as tqdm

In [4]:
# Look for all csv files
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [5]:
# Note that if you run this multiple times it will add combined_csv back to the csv file.
# ! Run this only once !
if not os.path.exists("combined_csv.csv"):
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
    combined_csv.to_csv("combined_csv.csv", index=False, encoding='utf-8-sig')


In [6]:
data = pd.read_csv(r'combined_csv.csv')
data

,Txhash,ContractAddress,ContractName
0,0xc439780ceb4726c5f1827d3f4a5d4c84b4deab83de5b...,0xf8D631FC499276F794f8D5b8F462C5b11A1d1Eef,StandardERC20
1,0xfda7fab6b89cb7a8fd827c4da118877ee5786d798845...,0x63c867e40ce5c278d153149060ef9b1d8a58c7ff,EthereumInu
2,0x066accd2a165a7b8a3d9f881f148d3a447ad047c17e4...,0x687c7cb36bb610bae16d2b4ec0f6584f29126767,PEPECOIN
3,0x0ff96b116a4a1cc9dd5fdb9e841e9e745e0126ef3a0f...,0x9287AEFe51047Ef43F68612f80BAC17745b23AeC,XRIToken
4,0x25f9d0716a5f99517b5b5ed7d78a92d007a3770363d1...,0xaebfe92a9f5e874bd680946b3a394e5165072569,ERC721Helper
...,...,...,...
63094,0x79a9f27073f953b832fd511015cc5f75a7eeac747e23...,0x6A858592fC4cBdf432Fc9A1Bc8A0422B99330bdF,DSValue
63095,0x311ae8809d6b7313f4d2b19b9bed94714ddeca13d141...,0x3f90622f3df1fe37e6233a8446a835ab032bb941,Registry
63096,0x021e9779d067aee34aa2ab93298f824f27d4b58c8e30...,0x2ce5a05aa98509632aa04a036337d0f8305e9e64,CatsCorporation
63097,0x54e6c6da32c28c17d818595d096f2fc927a8c7bc352a...,0x122Fb3Cb3f7939f58C808c0f267Dfa0eCAd0174C,NoumenaFloral


In [7]:
import requests
# Import your own apikey
apikey = '5NH6QCI59NUB74G9FY712QEHGIHZ4B4BBR'

In [8]:
def api_call(row):
    val = ""
    contractaddress = data['ContractAddress'][row]
    name = data['ContractName'][row]
    url = f'https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contractaddress}&apikey={apikey}'
    response = requests.get(url)
    response.raise_for_status()
    if response.status_code != 204:

        try:
            result = response.json()['result']
            val = result[0].get('SourceCode')
        except ValueError:
         val = "Value Error"

    return name, contractaddress, val


    #
    # call_count = val.count('.call(')
    # send_count = val.count('.send(')
    # transfer_count = val.count('.transfer(')


In [9]:
def write_to_csv(data, filename):
    with open(filename, 'w') as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerow(['ContractName','ContractAddress','Source'])
        for row in data:
            writer.writerows(row)

In [10]:
from multiprocessing import Pool, cpu_count
from p_tqdm import p_map
def multi_call(func, args):
    results = []
    with Pool(cpu_count()) as p:
        completed = p_map(func, args, **{"num_cpus":cpu_count()})
    results.append(completed)
    return results

In [11]:
def run_apply_async_multiprocessing(func, argument_list, num_processes):

    pool = Pool(processes=num_processes)

    jobs = [pool.apply_async(func=func, args=(*argument,)) if isinstance(argument, tuple) else pool.apply_async(func=func, args=(argument,)) for argument in argument_list]
    pool.close()
    result_list_tqdm = []
    for job in tqdm.tqdm(jobs):
        print(job.get()[1])

        result_list_tqdm.append(dict)

    return result_list_tqdm

In [12]:
import random
argument_list = [i for i in range(len(data))]
arg_list_1 = argument_list[:10000]
arg_list_2 = argument_list[10000:20000]
arg_list_3 = argument_list[20000:30000]
arg_list_4 = argument_list[30000:40000]
arg_list_5 = argument_list[40000:50000]
arg_list_6 = argument_list[50000:]

In [11]:
# Result 1
result1 = multi_call(api_call, arg_list_1)
write_to_csv(result1, 'contracts_1.csv')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [14]:
# Result 2
result2 = multi_call(api_call, arg_list_2)
write_to_csv(result2, 'contracts_2.csv')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [15]:
# Result 3
result3 = multi_call(api_call, arg_list_3)
write_to_csv(result3, 'contracts_3.csv')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [16]:
# Result 4
result4 = multi_call(api_call, arg_list_4)
write_to_csv(result4, 'contracts_4.csv')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
# Result 5
result5 = multi_call(api_call, arg_list_5)
write_to_csv(result5, 'contracts_5.csv')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
# Result 6
result6 = multi_call(api_call, arg_list_6)
write_to_csv(result6, 'contracts_6.csv')

  0%|          | 0/13099 [00:00<?, ?it/s]

In [14]:
all_contracts = [i for i in glob.glob('contracts*.{}'.format(extension))]

In [15]:
if not os.path.exists("all_contracts.csv"):
    contract_combo = pd.concat([pd.read_csv(f) for f in all_contracts])
    contract_combo.to_csv("all_contracts.csv", index=False, encoding='utf-8-sig')

,ContractName,ContractAddress,Source
0,Swapper,0xb4e5baff059c5ce3a0ee7ff8e9f16ca9dd91f1fe,pragma solidity 0.5.15;\r\npragma experimental...
1,blockrecipes,0xe065fbcba1f87abcd2d8538675d68b4a6ebabb33,// SPDX-License-Identifier: MIT\r\n\r\npragma ...
2,UBXTDistribution,0x07C01aa90BfdA9Aad888b464579920fdEec924B5,// SPDX-License-Identifier: MIT\n\npragma soli...
3,HPToken,0x00d07c53e70338c376cf6ab2a5218d8643115084,"{{\r\n ""language"": ""Solidity"",\r\n ""sources""..."
4,Muze,0x2b15d5857f4b1e247ae0335a7f3c1ec9148cfd1d,/**\r\nnothing to do with starlink and starlin...
...,...,...,...
63094,KCompound,0xAC053B14aF8493628a868Be1c6876Ec12889a424,"{{\r\n ""language"": ""Solidity"",\r\n ""settings..."
63095,JITUCompound,0x9e43efD070D8E3F8427862A760a37D6325821288,"{{\r\n ""language"": ""Solidity"",\r\n ""settings..."
63096,BABYCUBAN,0xd21730353086b0d6c34656ca74f4e4295dc9b8e0,// SPDX-License-Identifier: MIT\r\n\r\npragma ...
63097,RedEthRedemption,0xabb1282325fcbcb26098904a073f21de67a472a7,/*\r\n___________________ ______________...


In [13]:
contract_c = pd.read_csv(r'all_contracts.csv')
contract_c

,ContractName,ContractAddress,Source
0,Swapper,0xb4e5baff059c5ce3a0ee7ff8e9f16ca9dd91f1fe,pragma solidity 0.5.15;\r\npragma experimental...
1,blockrecipes,0xe065fbcba1f87abcd2d8538675d68b4a6ebabb33,// SPDX-License-Identifier: MIT\r\n\r\npragma ...
2,UBXTDistribution,0x07C01aa90BfdA9Aad888b464579920fdEec924B5,// SPDX-License-Identifier: MIT\n\npragma soli...
3,HPToken,0x00d07c53e70338c376cf6ab2a5218d8643115084,"{{\r\n ""language"": ""Solidity"",\r\n ""sources""..."
4,Muze,0x2b15d5857f4b1e247ae0335a7f3c1ec9148cfd1d,/**\r\nnothing to do with starlink and starlin...
...,...,...,...
63094,KCompound,0xAC053B14aF8493628a868Be1c6876Ec12889a424,"{{\r\n ""language"": ""Solidity"",\r\n ""settings..."
63095,JITUCompound,0x9e43efD070D8E3F8427862A760a37D6325821288,"{{\r\n ""language"": ""Solidity"",\r\n ""settings..."
63096,BABYCUBAN,0xd21730353086b0d6c34656ca74f4e4295dc9b8e0,// SPDX-License-Identifier: MIT\r\n\r\npragma ...
63097,RedEthRedemption,0xabb1282325fcbcb26098904a073f21de67a472a7,/*\r\n___________________ ______________...


In [92]:
import math
def get_counts(db):
    complete_list = []
    for row in tqdm.tqdm(range(len(db))):
        calls = 0
        transfers = 0
        sends = 0
        if not pd.isnull(db['Source'][row]):
            if '.call.value' in db['Source'][int(row)]:
                calls+=1
            if '.transfer' in db['Source'][int(row)]:
                transfers+=1
            if '.send' in db['Source'][int(row)]:
                sends+=1
        complete_list.append([(db['ContractName'][row], db['ContractAddress'][row], str(calls), str(transfers), str(sends))])
    return complete_list


In [93]:
def write_to_csv2(data_f, filename):
    with open(filename, 'w') as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerow(['ContractName','ContractAddress','Call','Transfer','Send'])
        for row in data_f:
            writer.writerows(row)

In [94]:
final = get_counts(contract_c)

100%|██████████| 63099/63099 [00:09<00:00, 6318.31it/s]


In [95]:
write_to_csv2(final,'analytics.csv')

In [86]:
test = multi_call(api_call, [1])

  0%|          | 0/1 [00:00<?, ?it/s]

In [90]:
print(type(test[0][0]))
print(type(test))

<class 'tuple'>
<class 'list'>


In [91]:
print(test[0][0])

('EthereumInu', '0x63c867e40ce5c278d153149060ef9b1d8a58c7ff', 'pragma solidity ^0.5.0;\r\n\r\n// ----------------------------------------------------------------------------\r\n// ERC Token Standard #20 Interface\r\n//\r\n// ----------------------------------------------------------------------------\r\ncontract ERC20Interface {\r\n    function totalSupply() public view returns (uint);\r\n    function balanceOf(address tokenOwner) public view returns (uint balance);\r\n    function allowance(address tokenOwner, address spender) public view returns (uint remaining);\r\n    function transfer(address to, uint tokens) public returns (bool success);\r\n    function approve(address spender, uint tokens) public returns (bool success);\r\n    function transferFrom(address from, address to, uint tokens) public returns (bool success);\r\n\r\n    event Transfer(address indexed from, address indexed to, uint tokens);\r\n    event Approval(address indexed tokenOwner, address indexed spender, uint t